In [198]:
#Basic working model using collaborative filtering with watched, this is just removing the watched movies from list and rating genres from watched history

In [199]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [200]:
# Sample user data
user_data = {
    'id': [1, 2, 3],
    'happy_movie': [['Drama', 'Comedy'], ['Romance', 'Family'], ['Adventure']],
    'sad_movie': [['Drama', 'Thriller'], ['Crime'], ['Horror', 'Science Fiction']],
    'neutral_movie': [['Comedy'], ['Action', 'Adventure'], ['Crime']],
    'fav_movie': ['The Godfather', "Schindler's List", 'The Shawshank Redemption']
}
user = pd.DataFrame(user_data)

In [201]:

# Load movie data from the specified file
file_path = '/content/drive/MyDrive/final project sample data/tmdb2024.csv'

In [202]:
minfo = pd.read_csv(file_path)

In [203]:
minfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359137 entries, 0 to 359136
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    359137 non-null  int64  
 1   id            359137 non-null  int64  
 2   title         359137 non-null  object 
 3   vote_average  359137 non-null  float64
 4   vote_count    359137 non-null  int64  
 5   imdb_id       358986 non-null  object 
 6   popularity    359137 non-null  float64
 7   poster_path   359046 non-null  object 
 8   Genres        359137 non-null  object 
 9   Release_Year  359137 non-null  int64  
 10  Release_Era   359137 non-null  object 
dtypes: float64(2), int64(4), object(5)
memory usage: 30.1+ MB


In [204]:
minfo

,Unnamed: 0,id,title,vote_average,vote_count,imdb_id,popularity,poster_path,Genres,Release_Year,Release_Era
0,0,27205,Inception,8.364,34495,tt1375666,83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,"['Action', 'Science Fiction', 'Adventure']",2010,mid
1,1,157336,Interstellar,8.417,32571,tt0816692,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,"['Adventure', 'Drama', 'Science Fiction']",2014,mid
2,2,155,The Dark Knight,8.512,30619,tt0468569,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,"['Drama', 'Action', 'Crime', 'Thriller']",2008,old
3,3,19995,Avatar,7.573,29815,tt0499549,79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",2009,old
4,4,24428,The Avengers,7.710,29166,tt0848228,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,"['Science Fiction', 'Action', 'Adventure']",2012,mid
...,...,...,...,...,...,...,...,...,...,...,...
359132,1019194,673471,Atanker Choya,0.000,0,tt10452806,0.600,/zMVHYoIzXe25rtImORhU5po0rk1.jpg,"['Adventure', 'Fantasy', 'Horror']",2019,mid
359133,1019200,673450,Pappa Lap 'n Verhaal van n pa en sy dogter,0.000,0,tt2027187,0.600,/oz0l9ZkVBMdFb9gTaE1TGk6T2of.jpg,['Drama'],1971,old
359134,1019201,673453,Doctor Who: The Pertwee Years,0.000,0,tt0297086,0.600,/lrSEDtfHMzFsRCXEuOjw8oiwLVh.jpg,['Documentary'],1992,old
359135,1019202,673455,The Taiga Troopers,0.000,0,tt0471935,0.627,/3jxhKTtZUwagmtEiv6tGaR5ZIuF.jpg,['Drama'],1966,old


In [205]:


minfo.rename(columns={'id': 'Movie_id'}, inplace=True)


movie_genres = minfo[['Movie_id', 'title', 'Genres', 'popularity', 'vote_average', 'vote_count','Release_Era']]





In [206]:
# Sample watched data
watched_data = {
    'userid': [1, 2, 3, 1, 2],
    'movieid': [238, 278, 240, 19404, 122],
    'rating_given': [8.0, 5.5, 9.5, 7.0, 6.5],
    'watching_mood': ['Happy', 'Sad', 'Neutral', 'Happy', 'Neutral']
}
watched = pd.DataFrame(watched_data)

In [207]:
movie

"Metallica: Masters of Metal (Genres: [, ', D, o, c, u, m, e, n, t, a, r, y, ', ,,  , ', M, u, s, i, c, ', ])"

In [208]:
movie_genres

,Movie_id,title,Genres,popularity,vote_average,vote_count,Release_Era
0,27205,Inception,"['Action', 'Science Fiction', 'Adventure']",83.952,8.364,34495,mid
1,157336,Interstellar,"['Adventure', 'Drama', 'Science Fiction']",140.241,8.417,32571,mid
2,155,The Dark Knight,"['Drama', 'Action', 'Crime', 'Thriller']",130.643,8.512,30619,old
3,19995,Avatar,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",79.932,7.573,29815,old
4,24428,The Avengers,"['Science Fiction', 'Action', 'Adventure']",98.082,7.710,29166,mid
...,...,...,...,...,...,...,...
359132,673471,Atanker Choya,"['Adventure', 'Fantasy', 'Horror']",0.600,0.000,0,mid
359133,673450,Pappa Lap 'n Verhaal van n pa en sy dogter,['Drama'],0.600,0.000,0,old
359134,673453,Doctor Who: The Pertwee Years,['Documentary'],0.600,0.000,0,old
359135,673455,The Taiga Troopers,['Drama'],0.627,0.000,0,old


In [209]:
# # prompt: Using dataframe movie: i want Genres to be reformatted with only list of name of genres, right now it has id and name and addtional text

# import ast

# # Create a copy of the dataframe
# movie_genres = movie.copy()

# # Convert the Genres column from string to list of dictionaries
# movie_genres['Genres'] = movie_genres['Genres'].apply(ast.literal_eval)

# # Extract the names of the genres from the list of dictionaries
# movie_genres['Genres'] = movie_genres['Genres'].apply(lambda x: [i['name'] for i in x])

# # Display the first 10 rows of the dataframe
# movie_genres.head(10)


In [210]:
# Assuming user, watched, and movie DataFrames are already defined as per your description.

# Merge the watched movies data with the movie details
watched_movies = pd.merge(watched, movie_genres, left_on='movieid', right_on='Movie_id')



In [211]:
watched_movies

,userid,movieid,rating_given,watching_mood,Movie_id,title,Genres,popularity,vote_average,vote_count,Release_Era
0,1,238,8.0,Happy,238,The Godfather,"['Drama', 'Crime']",158.448,8.707,18677,old
1,2,278,5.5,Sad,278,The Shawshank Redemption,"['Drama', 'Crime']",122.610,8.702,24649,old
2,3,240,9.5,Neutral,240,The Godfather Part II,"['Drama', 'Crime']",64.753,8.591,11293,old
3,1,19404,7.0,Happy,19404,Dilwale Dulhania Le Jayenge,"['Comedy', 'Drama', 'Romance']",33.575,8.552,4256,old
4,2,122,6.5,Neutral,122,The Lord of the Rings: The Return of the King,"['Adventure', 'Fantasy', 'Action']",99.276,8.474,22334,old


In [212]:
watched_movies['Genres']

0                    ['Drama', 'Crime']
1                    ['Drama', 'Crime']
2                    ['Drama', 'Crime']
3        ['Comedy', 'Drama', 'Romance']
4    ['Adventure', 'Fantasy', 'Action']
Name: Genres, dtype: object

In [213]:

from itertools import chain

# Assuming 'user' is your DataFrame containing user preferences.
# Sample data structure for 'user' DataFrame is provided in your initial description.

# Function to expand and tag mood to genres
def expand_user_genres(user_df, mood_column):
    # Explode the DataFrame by the specified mood to handle multiple genres
    expanded_df = user_df[['id', mood_column]].explode(mood_column)
    # Assign the current mood as a new column
    expanded_df['mood'] = mood_column.split('_')[0].capitalize()
    # Rename the genre column to a generic 'genre' column
    expanded_df = expanded_df.rename(columns={mood_column: 'genre'})
    return expanded_df

# Process each mood
happy_genres = expand_user_genres(user, 'happy_movie')
sad_genres = expand_user_genres(user, 'sad_movie')
neutral_genres = expand_user_genres(user, 'neutral_movie')

# Concatenate all moods together
all_genres = pd.concat([happy_genres, sad_genres, neutral_genres])

# Optional: Reset index if you want a clean DataFrame index
all_genres.reset_index(drop=True, inplace=True)

all_genres.head()


,id,genre,mood
0,1,Drama,Happy
1,1,Comedy,Happy
2,2,Romance,Happy
3,2,Family,Happy
4,3,Adventure,Happy


In [214]:
# Remove single quotes from 'Genres' column
movie_genres['Genres'] = movie_genres['Genres'].apply(lambda x: [genre.replace("'", "") for genre in x])

# Display the first few rows of the dataframe
print(movie_genres.head())




   Movie_id            title  \
0     27205        Inception   
1    157336     Interstellar   
2       155  The Dark Knight   
3     19995           Avatar   
4     24428     The Avengers   

                                              Genres  popularity  \
0  [[, , A, c, t, i, o, n, , ,,  , , S, c, i, e, ...      83.952   
1  [[, , A, d, v, e, n, t, u, r, e, , ,,  , , D, ...     140.241   
2  [[, , D, r, a, m, a, , ,,  , , A, c, t, i, o, ...     130.643   
3  [[, , A, c, t, i, o, n, , ,,  , , A, d, v, e, ...      79.932   
4  [[, , S, c, i, e, n, c, e,  , F, i, c, t, i, o...      98.082   

   vote_average  vote_count Release_Era  
0         8.364       34495         mid  
1         8.417       32571         mid  
2         8.512       30619         old  
3         7.573       29815         old  
4         7.710       29166         mid  


<ipython-input-214-bf661d2fd654>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_genres['Genres'] = movie_genres['Genres'].apply(lambda x: [genre.replace("'", "") for genre in x])


In [215]:
movie_genres['Genres']

0         [[, , A, c, t, i, o, n, , ,,  , , S, c, i, e, ...
1         [[, , A, d, v, e, n, t, u, r, e, , ,,  , , D, ...
2         [[, , D, r, a, m, a, , ,,  , , A, c, t, i, o, ...
3         [[, , A, c, t, i, o, n, , ,,  , , A, d, v, e, ...
4         [[, , S, c, i, e, n, c, e,  , F, i, c, t, i, o...
                                ...                        
359132    [[, , A, d, v, e, n, t, u, r, e, , ,,  , , F, ...
359133                            [[, , D, r, a, m, a, , ]]
359134          [[, , D, o, c, u, m, e, n, t, a, r, y, , ]]
359135                            [[, , D, r, a, m, a, , ]]
359136          [[, , D, o, c, u, m, e, n, t, a, r, y, , ]]
Name: Genres, Length: 359137, dtype: object

In [216]:
import ast
from sklearn.preprocessing import MultiLabelBinarizer

In [217]:
from sklearn.preprocessing import MultiLabelBinarizer
import re

# Preprocess genre names to remove unwanted characters using .loc
movie_genres.loc[:, 'Genres'] = movie_genres['Genres'].apply(lambda x: [re.sub(r'[^a-zA-Z]', '', genre) for genre in x])

# Use MultiLabelBinarizer
mlb = MultiLabelBinarizer()
movie_genres_encoded = mlb.fit_transform(movie_genres['Genres'])
movie_genres_df = pd.DataFrame(movie_genres_encoded, columns=mlb.classes_, index=movie_genres.index)

# Display the resulting DataFrame
print(movie_genres_df)



           A  C  D  F  H  M  R  S  T  ...  l  m  n  o  r  s  t  u  v  y
0       1  1  0  0  1  0  0  0  1  0  ...  0  0  1  1  1  0  1  1  1  0
1       1  1  0  1  1  0  0  0  1  0  ...  0  1  1  1  1  0  1  1  1  0
2       1  1  1  1  0  0  0  0  0  1  ...  1  1  1  1  1  0  1  0  0  0
3       1  1  0  0  1  0  0  0  1  0  ...  0  0  1  1  1  1  1  1  1  1
4       1  1  0  0  1  0  0  0  1  0  ...  0  0  1  1  1  0  1  1  1  0
...    .. .. .. .. .. .. .. .. .. ..  ... .. .. .. .. .. .. .. .. .. ..
359132  1  1  0  0  1  1  0  0  0  0  ...  0  0  1  1  1  1  1  1  1  1
359133  1  0  0  1  0  0  0  0  0  0  ...  0  1  0  0  1  0  0  0  0  0
359134  1  0  0  1  0  0  0  0  0  0  ...  0  1  1  1  1  0  1  1  0  1
359135  1  0  0  1  0  0  0  0  0  0  ...  0  1  0  0  1  0  0  0  0  0
359136  1  0  0  1  0  0  0  0  0  0  ...  0  1  1  1  1  0  1  1  0  1

[359137 rows x 28 columns]


In [218]:
import ast
from sklearn.preprocessing import MultiLabelBinarizer

def safe_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return []

# Apply the safe literal evaluation function using .loc
movie_genres.loc[:, 'Genres'] = movie_genres['Genres'].apply(safe_literal_eval)







In [219]:
print(movie_genres['Genres'].head())


0    []
1    []
2    []
3    []
4    []
Name: Genres, dtype: object


In [220]:
movie_genres

,Movie_id,title,Genres,popularity,vote_average,vote_count,Release_Era
0,27205,Inception,[],83.952,8.364,34495,mid
1,157336,Interstellar,[],140.241,8.417,32571,mid
2,155,The Dark Knight,[],130.643,8.512,30619,old
3,19995,Avatar,[],79.932,7.573,29815,old
4,24428,The Avengers,[],98.082,7.710,29166,mid
...,...,...,...,...,...,...,...
359132,673471,Atanker Choya,[],0.600,0.000,0,mid
359133,673450,Pappa Lap 'n Verhaal van n pa en sy dogter,[],0.600,0.000,0,old
359134,673453,Doctor Who: The Pertwee Years,[],0.600,0.000,0,old
359135,673455,The Taiga Troopers,[],0.627,0.000,0,old


In [221]:
# Now, you can use MultiLabelBinarizer
mlb = MultiLabelBinarizer()
movie_genres_encoded = mlb.fit_transform(movie_genres['Genres'])
movie_genres_df = pd.DataFrame(movie_genres_encoded, columns=mlb.classes_, index=movie_genres['Movie_id'])

In [222]:
movie_genres_encoded

array([], shape=(359137, 0), dtype=int64)

In [223]:
movie_genres_df

""
Movie_id
27205
157336
155
19995
24428
...
673471
673450
673453


In [224]:
# def recommend_movies_with_ratings_and_mood(user_id, current_mood, top_n=5):
#     # Get genres the user prefers in the current mood
#     mood_genres = all_genres[(all_genres['id'] == user_id) & (all_genres['mood'] == current_mood)]
#     user_pref_genres = mood_genres['genre'].unique()

#     # Get movies watched and rated by the user in the specified mood
#     mood_watched_movies = watched_movies[(watched_movies['userid'] == user_id) & (watched_movies['watching_mood'].str.capitalize() == current_mood)]

#     # Calculate average ratings for genres in this mood

#     mood_watched_movies_details = mood_watched_movies.merge(movie_genres, left_on='movieid', right_index=True)

#     # Adjusted section to use 'Genres_x'
#     genre_ratings = {}
#     for genre in user_pref_genres:
#         # Here we filter using 'Genres_x'
#         genre_movies = mood_watched_movies_details[mood_watched_movies_details['Genres_x'].apply(lambda x: genre in x)]
#         avg_rating = genre_movies['rating_given'].mean()
#         genre_ratings[genre] = avg_rating if not np.isnan(avg_rating) else 0


#     # Rank genres by average rating
#     ranked_genres = sorted(genre_ratings, key=genre_ratings.get, reverse=True)

#     # Encode user preferences weighted by average genre rating
#     user_pref_encoded = mlb.transform([ranked_genres])

#     # Calculate similarity between weighted user preferences and each movie's genres
#     similarity = cosine_similarity(user_pref_encoded, movie_genres_encoded)

#     # Exclude movies already watched
#     watched_movie_ids = watched_movies[watched_movies['userid'] == user_id]['movieid'].unique()
#     potential_movies = movie_genres[~movie_genres.index.isin(watched_movie_ids)]

#     # Get top N recommended movie IDs based on similarity scores, excluding watched movies
#     recommended_movie_ids = similarity.argsort()[0][-top_n:][::-1]

#     # Ensure correct indexing and data type consistency for recommended_movie_ids
#     recommended_movie_ids = [potential_movies.index[i] for i in recommended_movie_ids if i < len(potential_movies)]

#     # Map movie IDs back to movie titles and genres
#     recommended_movies_info = movie_genres.loc[recommended_movie_ids][['title', 'Genres']]

#     # Create a more readable output format
#     recommended_movies_output = recommended_movies_info.apply(lambda x: f"{x['title']} (Genres: {', '.join(x['Genres'])})", axis=1)

#     return recommended_movies_output.tolist()

# # Example usage
# recommended_movies = recommend_movies_with_ratings_and_mood(1, 'Happy', 5)
# for movie in recommended_movies:
#     print(movie)


In [225]:
def recommend_movies_with_weighted_genres(user_id, current_mood, top_n=5):
    mood_genres = all_genres[(all_genres['id'] == user_id) & (all_genres['mood'] == current_mood)]
    user_pref_genres = mood_genres['genre'].unique()

    # Check for mood-specific watched movies
    mood_watched_movies = watched_movies[(watched_movies['userid'] == user_id) & (watched_movies['watching_mood'].str.capitalize() == current_mood)]

    genre_ratings = {}
    if not mood_watched_movies.empty:
        # Calculate average ratings for genres from mood-specific watched movies
        mood_watched_movies_details = mood_watched_movies.merge(movie_genres, left_on='movieid', right_index=True)
        for genre in user_pref_genres:
            genre_movies = mood_watched_movies_details[mood_watched_movies_details['Genres_x'].apply(lambda x: genre in x)]
            avg_rating = genre_movies['rating_given'].mean()
            genre_ratings[genre] = avg_rating if not np.isnan(avg_rating) else 0
        # Normalize ratings to use as weights
        max_rating = max(genre_ratings.values(), default=1)
        genre_weights = {genre: rating / max_rating for genre, rating in genre_ratings.items()}
    else:
        genre_weights = {genre: 1 for genre in user_pref_genres}  # Default weight if no ratings

    # Encode movie genres and user preferences
    mlb = MultiLabelBinarizer()
    movie_genres_encoded = mlb.fit_transform(movie_genres['Genres'])
    genre_prefs_encoded = mlb.transform([user_pref_genres])

    # Calculate similarity with weighted genres
    similarity_scores = cosine_similarity(genre_prefs_encoded, movie_genres_encoded)[0]
    weighted_similarity_scores = np.zeros_like(similarity_scores)

    for i, genre in enumerate(mlb.classes_):
        if genre in genre_weights:
            weighted_similarity_scores += similarity_scores * movie_genres_encoded[:, i] * genre_weights[genre]

    # Exclude watched movies
    watched_movie_ids = watched_movies[watched_movies['userid'] == user_id]['movieid'].unique()
    unwatched_indices = [i for i, movie_id in enumerate(movie_genres.index) if movie_id not in watched_movie_ids]

    # Recommend based on weighted similarity scores
    unwatched_similarity_scores = weighted_similarity_scores[unwatched_indices]
    top_indices = np.argsort(unwatched_similarity_scores)[-top_n:][::-1]

    recommended_movie_ids = [movie_genres.index[unwatched_indices[i]] for i in top_indices]
    recommended_movies_info = movie_genres.loc[recommended_movie_ids]
    recommended_movies_output = recommended_movies_info.apply(lambda x: f"{x['title']} (Genres: {', '.join(x['Genres'])})", axis=1).tolist()

    return recommended_movies_output

# Example usage
recommended_movies = recommend_movies_with_weighted_genres(1, 'Happy', 10)
for movie in recommended_movies:
    print(movie)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['Comedy', 'Drama'] will be ignored
  warnings.warn(


ValueError: Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required by check_pairwise_arrays.

In [ ]:
# Example: Recommend movies for user 1 in a "Happy" mood, including genres
recommended_movies = recommend_movies_with_weighted_genres(1, 'Happy', 10)
for movie in recommended_movies:
    print(movie)

In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(1, 'Sad', 10)
for movie in recommended_movies:
    print(movie)

In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(1, 'Neutral', 10)
for movie in recommended_movies:
    print(movie)

In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(2, 'Happy', 200)
for movie in recommended_movies:
    print(movie)


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(2, 'Sad', 10)
for movie in recommended_movies:
    print(movie)

In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(2, 'Neutral', 10)
for movie in recommended_movies:
    print(movie)

In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(3, 'Happy', 10)
for movie in recommended_movies:
    print(movie)

In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(3, 'Sad', 10)
for movie in recommended_movies:
    print(movie)

In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(3, 'Neutral', 10)
for movie in recommended_movies:
    print(movie)

In [ ]:
user

Relevant code ends here

In [ ]:
minfo.info()

In [ ]:
minfo